In [1]:
import Algorithms as alg
import numpy as np
import pandas as pd
import csv

df = pd.read_pickle('dataframes/df_mitdb.pkl')
df.head()

,ECG,True_peak,Peak_type
100,"[-0.145, -0.145, -0.145, -0.145, -0.145, -0.14...","[18, 77, 370, 662, 946, 1231, 1515, 1809, 2044...","[+, N, N, N, N, N, N, N, A, N, N, N, N, N, N, ..."
101,"[-0.345, -0.345, -0.345, -0.345, -0.345, -0.34...","[7, 83, 396, 711, 1032, 1368, 1712, 2036, 2349...","[+, N, N, N, N, N, N, N, N, N, N, N, N, N, N, ..."
102,"[-0.2, -0.2, -0.2, -0.2, -0.2, -0.2, -0.2, -0....","[68, 136, 410, 697, 989, 1305, 1614, 1911, 220...","[+, /, /, /, /, /, /, /, /, /, /, /, /, /, /, ..."
103,"[-0.375, -0.375, -0.375, -0.375, -0.375, -0.37...","[21, 265, 575, 876, 1180, 1482, 1795, 2127, 24...","[+, N, N, N, N, N, N, N, N, N, N, N, N, N, N, ..."
104,"[-0.15, -0.15, -0.15, -0.15, -0.15, -0.15, -0....","[36, 73, 316, 615, 901, 1188, 1477, 1749, 2062...","[+, /, /, /, /, /, f, f, /, /, /, /, /, f, f, ..."


In [2]:
R_SYMBOLS = ['N', 'V', 'L', 'R', '/', 'f', 'A', 'E', 'Q', 'F', 'j', 'J', 'a', 'S', 'e']
DETECTION_X_RANGE = 58

def binary_classifier(r_peaks_annotated, found_r_peaks):
    """
    Funkcja pochodzi z pliku "tester.py"
    """
    t_pos = 0
    if len(r_peaks_annotated) > 0:
        bitmap_len = r_peaks_annotated[-1] + DETECTION_X_RANGE
        found_bitmap = np.zeros(bitmap_len)
        for x in found_r_peaks:
            if x < bitmap_len:
                found_bitmap[x] = 1
        for x in r_peaks_annotated:
            left_thres = max(0, x - DETECTION_X_RANGE)
            right_thres = min(bitmap_len, x + DETECTION_X_RANGE + 1)
            found = False
            for i in range(left_thres, right_thres):
                if found_bitmap[i]:
                    found = True
                    found_bitmap[i] = 0
                    break
            if found:
                t_pos += 1
            # else:
            #     print('x: ' + str(x) + ', time: ' +  sample_time_converter.sample_to_time(x))
    else:
        t_pos = 0
    f_neg = len(r_peaks_annotated) - t_pos
    f_pos = len(found_r_peaks) - t_pos
    # print('t_pos:', t_pos, 'f_pos:', f_pos, 'f_neg: ', f_neg)
    return t_pos, f_pos, f_neg

def binary_classifier_2(r_peaks_annotated, found_r_peaks):
    """
    Funkcja pochodzi z pliku "tester_start_algorithm.py"
    """
    TP = []
    FP = []
    FN = []
    if len(r_peaks_annotated) > 0:
        bitmap_len = r_peaks_annotated[-1] + DETECTION_X_RANGE
        found_bitmap = np.zeros(bitmap_len)
        for x in found_r_peaks:
            if x < bitmap_len:
                found_bitmap[x] = 1
        for x in r_peaks_annotated:
            left_thres = max(0, x - DETECTION_X_RANGE)
            right_thres = min(bitmap_len, x + DETECTION_X_RANGE + 1)
            found = False
            for i in range(left_thres, right_thres):
                if found_bitmap[i]:
                    found = True
                    found_bitmap[i] = 0
                    TP.append(i)
                    break
            if not found:
                FP.append(x)
    for sample in found_r_peaks:
        if sample not in TP:
            FN.append(sample)
    return TP, FP, FN


window = (50, 50)

def find_TP_FP_FN(true_peaks, detected, window):
    TP = 0
    for peak in true_peaks:
        mask = (detected >= peak-window[0]) & (detected <= peak+window[0])
        is_detected = detected[mask]

        if len(is_detected) >= 1:
            TP += 1

    FN = len(true_peaks) - TP
    FP = len(detected) - TP

    return TP, FP, FN
    


In [5]:
# algorithm = alg.alg1_spanish
# algorithm = alg.alg3_iranian
algorithm = alg.Alg5_Turkish

def evaluate_algorithm(df, algorithm, write_csv=False, show_results=True, name='Output.csv'):

    if show_results==True:
        print('tape', 'TB ', '  TP', '   FN', 'FP', 'Se', '    +P')

    if write_csv==True:
        with open(name, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['tape', 'TB', 'TP', 'FN', 'FP', 'Se', '+P'])

    total_TB = 0
    total_TP = 0
    total_FP = 0
    total_FN = 0

    for tape in df.index:
        signal = df.at[tape, 'ECG']
        true_peaks_all = np.array(df.at[tape, 'True_peak'])
        annotations = np.array(df.at[tape, 'Peak_type'])
    
        true_peaks = []
        for peak, annot in zip(true_peaks_all, annotations):
            if annot in R_SYMBOLS:
                true_peaks.append(peak)

        true_peaks = np.array(true_peaks)
        detected = np.array(algorithm(signal))

        TP, FP, FN = binary_classifier(true_peaks, detected)
        
        Se = TP/(TP+FN)
        P = TP/(TP+FP)

        if show_results==True:
            print(f'{tape}, {TP+FN}, {TP}, {FN}, {FP}, {round(100*Se, 2)}, {round(100*P, 2)}')

        if write_csv==True:
            with open(name, mode='a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow([tape, TP+FN, TP, FN, FP, round(100*Se, 2), round(100*P, 2)])

        total_TB += TP + FN
        total_TP += TP
        total_FP += FP
        total_FN += FN

    total_Se = total_TP/(total_TP+total_FN)
    total_P = total_TP/(total_TP+total_FP)

    if show_results==True:
            print(f'TOTAL, {total_TB}, {total_TP}, {total_FN}, {total_FP}, {round(100*total_Se, 2)}, {round(100*total_P, 2)}')

    if write_csv==True:
        with open(name, mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['TOTAL', total_TB, total_TP, total_FN, total_FP,  
                            round(100*total_Se, 2), round(100*total_P, 2)])

    return {'TP': total_TP, 'FN': total_FN, 'FP': total_FP,
            'Se': round(100*total_Se, 2), '+P': round(100*total_P, 2)}



In [6]:
results = evaluate_algorithm(df, algorithm, write_csv=False)

tape TB    TP    FN FP Se     +P
100, 2273, 2271, 2, 1, 99.91, 99.96
101, 1865, 1860, 5, 4, 99.73, 99.79
102, 2187, 2186, 1, 1, 99.95, 99.95
103, 2084, 2083, 1, 1, 99.95, 99.95
104, 2229, 2203, 26, 20, 98.83, 99.1
105, 2572, 2538, 34, 8, 98.68, 99.69
106, 2027, 2018, 9, 12, 99.56, 99.41
107, 2137, 2135, 2, 1, 99.91, 99.95
108, 1763, 1593, 170, 173, 90.36, 90.2
109, 2532, 2530, 2, 1, 99.92, 99.96
111, 2124, 2123, 1, 2, 99.95, 99.91
112, 2539, 2539, 0, 0, 100.0, 100.0
113, 1795, 1794, 1, 1, 99.94, 99.94
114, 1879, 1871, 8, 9, 99.57, 99.52
115, 1953, 1953, 0, 0, 100.0, 100.0
116, 2412, 2394, 18, 5, 99.25, 99.79
117, 1535, 1532, 3, 4, 99.8, 99.74
118, 2278, 2272, 6, 8, 99.74, 99.65
119, 1987, 1986, 1, 2, 99.95, 99.9
121, 1863, 1855, 8, 14, 99.57, 99.25
122, 2476, 2476, 0, 0, 100.0, 100.0
123, 1518, 1515, 3, 2, 99.8, 99.87
124, 1619, 1617, 2, 2, 99.88, 99.88
200, 2601, 2578, 23, 18, 99.12, 99.31
201, 1963, 1883, 80, 41, 95.92, 97.87
202, 2136, 2111, 25, 6, 98.83, 99.72
203, 2980, 2232, 748,